In [1]:
import pandas as pd
import geopandas as gpd
import json
import plotly as pl
import plotly.express as px

In [10]:
# get data from file
def plot_month(month):
    month_data = pd.read_csv('data_processing_income\clean data\\' + month + '.csv')
    
    # define specific value for median so that colours can be plotted. Also, correct for the different ranges in files. Should automate this!!

    medians_range_to_value = {
        'Less Than $5,000': 2500,
        '5,000 To 7,499': 6500,
        '7,500 To 9,999': 8750,
        '10,000 To 12,499': 11500,
        '12,500 To 14,999': 13750,
        '15,000 To 19,999': 17500,
        '20,000 To 24,999': 22500,
        '25,000 To 29,999': 27500,
        '30,000 To 34,999': 32500,
        '35,000 To 39,999': 37500,
        '40,000 To 49,999': 45000,
        '50,000 To 59,999': 55000,
        '60,000 To 74,999': 67500,
        '75,000 To 99,999': 87500,
        '100,000 To 149,999': 125000,
        '150,000 or More': 150000,
        'Less than $25,000': 12500,
        '$25,000 - $34,999': 30000,
        '$35,000 - $49,999': 42500,
        '$50,000 - $74,999': 62500,
        '$75,000 - $99,999': 87500,
        '$100,000 - $149,999': 125000,
        '$150,000 - $199,999': 175000,
        '$200,000 and above': 200000
    }

    # replace range with specific value
    month_data['median_income_' + month] = month_data['median_income_' + month].replace(medians_range_to_value).astype(float)

    return month_data

In [11]:
plot_month('apr')
# plot_month('w1_0423_0505.csv')

,Selected Geographies,abbreviations,FIPS_code,median_income_apr
0,Alabama,AL,1,55000.0
1,Alaska,AK,2,67500.0
2,Arizona,AZ,4,67500.0
3,Arkansas,AR,5,45000.0
4,California,CA,6,67500.0
5,Colorado,CO,8,87500.0
6,Connecticut,CT,9,87500.0
7,Delaware,DE,10,67500.0
8,District of Columbia,DC,11,87500.0
9,Florida,FL,12,55000.0


In [ ]:
# reference https://wellsr.com/python/creating-python-choropleth-maps-with-plotly/

def plot_map(month):
    
    month_data = plot_month(month)

    fig = px.choropleth(month_data,
                        locations = 'abbreviations',
                        color = 'median_income_' + month,
                        hover_name = 'Selected Geographies',
                        locationmode = 'USA-states'
    )

    fig.update_layout(
        title_text = 'Median Incomes for ' + month,
        geo_scope = 'usa',
    )

    fig.show()
    return

In [17]:
list_of_files = ['jan',
                'feb',
                'mar',
                'apr',
                'w1_0423_0505',
                'w2_0507_0512',
                'w3_0514_0519'
                ]

for i in range(len(list_of_files)):
    plot_map(list_of_files[i])

# left to do:
# get it to work with dataset #2
# get them to be on the same file/time slider 
# change colours?

KeyError: 'median_income_w1_0423_0505'

In [4]:
plot_map('jan')

NameError: name 'plot_map' is not defined